In [1]:
import os, sys
import pandas as pd

notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [2]:
from scripts.data_loader import load_data
from scripts.data_loader import load_data_with_parse
from scripts.data_analysis import get_sentiment
from textblob import TextBlob
from scripts.data_visualization import plot_bivariate

In [3]:

aapl_data = load_data_with_parse("../data/yfinance_data/AAPL_historical_data.csv", parse_dates=['Date'])
news_data = load_data_with_parse("../data/raw_analyst_ratings.csv", parse_dates=['Date'])


In [4]:
news_data = news_data.rename(columns={'date': 'Date'})

In [5]:
aapl_stock_data = aapl_data.sort_values(by='Date').reset_index(drop=True)
news_data = news_data.sort_values(by='Date').reset_index(drop=True)

In [6]:
news_data = news_data.drop_duplicates(subset=['Date']).reset_index(drop=True)

In [7]:
news_data["Date"] = pd.to_datetime(news_data["Date"], format="mixed", utc=True).dt.date

In [8]:
combined_data = pd.merge(aapl_stock_data[['Date', 'Close']], news_data, on='Date', how='inner')

In [9]:
combined_data

,Date,Close,Unnamed: 0,headline,url,publisher,stock


In [10]:
combined_data['Sentiment'] = combined_data['headline'].apply(get_sentiment)

In [11]:
combined_data['Daily_Return'] = combined_data['Close'].pct_change()
combined_data = combined_data.dropna(subset=['Daily_Return']).reset_index(drop=True)

In [12]:
correlation = combined_data['Sentiment'].corr(combined_data['Daily_Return'])
print(f"Correlation between News Sentiment and Stock Returns: {correlation:.2f}")

plot_bivariate(data=combined_data, x_column='Sentiment', y_column='Daily_Return', title='Visualize the relationship')

Correlation between News Sentiment and Stock Returns: nan


TypeError: plot_bivariate() got an unexpected keyword argument 'x'